# Consigna

Link al parcial: https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqsotjd4yk

Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto. 

Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017. 

In [ ]:
from pyspark import SparkContext
#  Por si hay mas de un contexto de PySpark corriendo (por ejemplo, otro Notebook), esto para utilizar el mismo.
sc = SparkContext.getOrCreate()

In [ ]:
#  Creamos algunos datos para poder hacer el seguimiento de la resolución.
#  El resultado final debería ser ('Chevrolet', 'Sonic'), ('Ford', 'Cargo 712') y ('Honda', 'Hornet 160R').

datos_patentamientos = [
    ('MHG 100', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '2017-03-15'),
    ('MHG 101', 'Ford', 'Cargo 712', 2, 'camion', 'Chaco', '2017-03-19'),
    ('MHG 102', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '2017-04-01'),
    ('MHG 103', 'Fiat', 'Siena', 2, 'auto', 'Buenos Aires', '2017-04-02'),
    ('MHG 104', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '2017-04-02'),
    ('MHG 105', 'Fiat', 'Siena', 3, 'auto', 'Uruguay', '2017-04-03'),
    ('MHG 106', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '2017-04-05'),
    ('MHG 107', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '2017-04-17'),
    ('MHG 108', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '2017-04-19'),
    ('MHG 109', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '2017-04-19'),
    ('MHG 110', 'Ford', 'Cargo 712', 2, 'camion', 'Buenos Aires', '2017-04-19'),
    ('MHG 111', 'Fiat', 'Siena', 3, 'auto', 'Cordoba', '2017-04-20'),
    ('MHG 112', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '2017-04-21'),
    ('MHG 113', 'Fiat', 'Sedan', 2, 'auto', 'Buenos Aires', '2017-04-23'),
    ('MHG 114', 'Fiat', 'Sedan', 1, 'auto', 'Buenos Aires', '2017-04-24'),
    ('MHG 115', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '2017-04-25'),
    ('MHG 116', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '2017-04-25'),
    ('MHG 117', 'Ducati', 'SuperSport', 1, 'moto', 'Buenos Aires', '2017-04-26'),
    ('MHG 118', 'Scania', '420', 4, 'camion', 'Buenos Aires', '2017-04-26')
]

# Resolución

Nos piden filtrar por fecha y provincia. Luego agrupar por tipo_vehiculo, marca y modelo, y encontrar el vehículo más patentado por tipo_vehiculo.

Observación: Es importante notar que en __ningun__ caso vamos a realizar ordenamientos, ya que son operaciones muy costosas y no son necesarias para la resolución de este ejercicio.

### Paso 1

Cargamos los datos en un RDD, y luego aplicamos el filtro por las dos condiciones que nos indican en el enunciado. 

Además, nos quedamos con tuplas de la pinta `((tipo_vehiculo, (marca, modelo)), 1)` para poder realizar un reduceByKey en el paso siguiente y obtener la cantidad de patentamientos totales para cada vehículo.

Observación: vehículo == (marca, modelo)

In [ ]:
rdd_pt = sc.parallelize(datos_patentamientos)

In [ ]:
#  Header de rdd_pt: [patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha]

CAMPO_PROVINCIA = 5
CAMPO_FECHA = 6

CAMPO_MODELO = 2
CAMPO_MARCA = 1
CAMPO_TIPO_VEHICULO = 4

def organizar_clave(x):
    return ((x[CAMPO_TIPO_VEHICULO], 
                 (x[CAMPO_MARCA], x[CAMPO_MODELO])
            ), 1)

rdd_pt = rdd_pt.filter(lambda x: x[CAMPO_PROVINCIA] == 'Buenos Aires' and x[CAMPO_FECHA].startswith('2017-04')).map(organizar_clave)
rdd_pt.collect()

### Paso 2

Haremos un `reduceByKey` para obtener el total de patentamientos de cada `(marca, modelo)` por `tipo_vehiculo`. 

Luego reorganizaremos los datos con `map` para tener tuplas del estilo `(tipo_vehiculo, ((marca, modelo), total_patentamientos)` lo cual será necesario para el paso final.

Observación: Notar que las claves por las que se agrupa son de la pinta `(tipo_vehiculo, (marca, modelo))`.

In [ ]:
def acomodar_modelo(x):
    return (x[0][0], (x[0][1], x[1]))

rdd_pt = rdd_pt.reduceByKey(lambda a,b: a+b).map(acomodar_modelo)
rdd_pt.collect()

### Paso 3

Finalmente aplicaremos otro `reduceByKey` para obtener el vehículo con mayor cantidad de patentamientos por tipo de vehículo.

Observación: Notar que `a[1]` es el segundo elemento de la tupla `((marca, modelo), total)`.

In [ ]:
rdd_pt = rdd_pt.reduceByKey(lambda a,b: a if a[1] > b[1] else b)
print("\n".join(rdd_pt.map(lambda x: x[0] + ': ' + " ".join(x[1][0])).collect()))